## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
#city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,3,Axim,GH,79.36,few clouds,4.8699,-2.2405,AXIM BEACH RESORT AND SPA
1,4,Touros,BR,78.80,few clouds,-5.1989,-35.4608,INN NEW HORIZON
2,5,Vaini,TO,86.00,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
3,7,Alofi,NU,86.00,clear sky,-19.0595,-169.9187,Taloa Heights
4,9,Rikitea,PF,79.09,broken clouds,-23.1203,-134.9692,Pension Maro'i


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,3,Axim,GH,79.36,few clouds,4.8699,-2.2405,AXIM BEACH RESORT AND SPA
1,4,Touros,BR,78.80,few clouds,-5.1989,-35.4608,INN NEW HORIZON
2,5,Vaini,TO,86.00,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
3,7,Alofi,NU,86.00,clear sky,-19.0595,-169.9187,Taloa Heights
4,9,Rikitea,PF,79.09,broken clouds,-23.1203,-134.9692,Pension Maro'i
5,10,Kirakira,SB,81.64,broken clouds,-10.4544,161.9205,SanBiz Lodge
6,12,Tirumullaivasal,IN,79.25,scattered clouds,11.2333,79.8333,Hajaali
7,16,Avarua,CK,84.20,few clouds,-21.2078,-159.7750,Paradise Inn
8,23,Meulaboh,ID,74.48,overcast clouds,4.1363,96.1285,Kost Pria
9,37,Victoria,HK,79.00,overcast clouds,22.2855,114.1577,Mini Hotel Central


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        260
City           260
Country        260
Max Temp       260
Description    260
Lat            260
Lng            260
Hotel Name     248
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows. LOOK HERE  (from 4.5.2)
clean_df=preferred_cities_df.dropna()



In [7]:
#check that the row was dropped.  It doesn't look like it.
clean_df.count()

City_ID        248
City           248
Country        248
Max Temp       248
Description    248
Lat            248
Lng            248
Hotel Name     248
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Axim,GH,79.36,few clouds,4.8699,-2.2405,
1,Touros,BR,78.80,few clouds,-5.1989,-35.4608,
2,Vaini,TO,86.00,broken clouds,-21.2000,-175.2000,
3,Alofi,NU,86.00,clear sky,-19.0595,-169.9187,
4,Rikitea,PF,79.09,broken clouds,-23.1203,-134.9692,
5,Kirakira,SB,81.64,broken clouds,-10.4544,161.9205,
6,Tirumullaivasal,IN,79.25,scattered clouds,11.2333,79.8333,
7,Avarua,CK,84.20,few clouds,-21.2078,-159.7750,
8,Meulaboh,ID,74.48,overcast clouds,4.1363,96.1285,
9,Victoria,HK,79.00,overcast clouds,22.2855,114.1577,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
# 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"]=hotels["results"][0]["name"]
    except: 
        print("Hotel not found")


In [10]:
    
hotels

{'html_attributions': [],
 'next_page_token': 'ATtYBwIVNhOV8IRDRvazAYAJ46GStfpmXNSGcGeyOmpcp88iDHryX6U3dqkMDxwrj_kUBYH2n_5jBcIsq1GcOrN2amN2-FISSdD4w3nsYi9MLAaOMtmT0NtICy2xpFaomAONqrWEegJ4agCjWgStn_gHIgoKwriT5gm-6cz2_kDzajr48PFWvh1Jovn4ip2p3rxBHQM8pqGEEM70Kemp5COBmlNaujdjWQKVN4jHhlNWsjdiKujHBTCKmOAGsDZS8L0VTFKrfL7RGOqzENnBzixw99_opeMvGZrYHv_gF1QeNkejDO4S7CF5JkJ7n4AHFDIPOZ_0BSSbcDcn65OuxIvv5tELSvEnWYf8b46_5WQtlJ5kYbYcZ9TIzusNGrRQBG4IVWyE6oxkmK6XltvZkEvRhIkkvJkdAjXn3cDGGxmIaayAl2LYmfMaYEkI',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': -31.4284079, 'lng': 152.909174},
    'viewport': {'northeast': {'lat': -31.4270259697085,
      'lng': 152.9106213802915},
     'southwest': {'lat': -31.4297239302915, 'lng': 152.9079234197085}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'name': 'Rydges Port Macquarie',
   'opening_hours': {'open_now': True},
   'photos': [{'height': 3744,
     'html_attributions': ['<

In [11]:
# 7. Drop the rows where there is no Hotel Name. RECHECK THIS not sure this is right.
hotel_df.count()

City           248
Country        248
Max Temp       248
Description    248
Lat            248
Lng            248
Hotel Name     248
dtype: int64

In [12]:
hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Axim,GH,79.36,few clouds,4.8699,-2.2405,AXIM BEACH RESORT AND SPA
1,Touros,BR,78.80,few clouds,-5.1989,-35.4608,INN NEW HORIZON
2,Vaini,TO,86.00,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
3,Alofi,NU,86.00,clear sky,-19.0595,-169.9187,Taloa Heights
4,Rikitea,PF,79.09,broken clouds,-23.1203,-134.9692,Pension Maro'i


In [13]:
# 8a. Create the output File (CSV)   Check output location...looks incorrect.
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))